In [680]:
# 201908이강훈 작성

## 키워드 선정
condition_list = [
   "유상증자결정",
   "최대주주",
    "투자판단",
    "조회공시",
   "단일판매ㆍ공급계약체결", 
   "특허권 취득",
  "영업실적 등에 대한 전망",
  "수시공시",
   "신규시설투자등",
   "소송",
   "거래정지",
   "영업정지",
   "관리종목",
   "매출액",
   "상장폐지",
   "중단",
   "타법인주식및출자증권처분결정",
   "연결재무제표 기준 영업(잠정)실적",
   "타법인주식및출자증권취득결정",
   "우선주의 보통주",
   "전환청구권",
   "영업(잠정)실적(공정공시)",
   "영업양수결정",
   "연결재무제표기준영업(잠정)실적",
    "주요사항보고서",
  "최대주주변경을수반하는주식양수도계약체결",
  "타법인 주식 및 출자증권 양수결정",
   "주식소각결정",
    "기술도입ㆍ이전ㆍ제휴 계약체결",
   "기타 경영사항"
    ]

## 1. 공시 크롤링

## 1-1 파라미터 설정

In [681]:
import urllib.request
import urllib.parse
import xlsxwriter
import os
import time
import sys
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re
from urllib import parse
import pandas as pd
import traceback
import requests
import numpy as np

#====================================================
#=================파라미터 설정 부분=====================
#====================================================
maxResults = '100'  ##15, 30, 60, 100
maxLinks = '10'
sort = 'date'
series = 'desc'
reportNamePopYn =  'N'
textCrpNm = '삼성전자' ##기업이름
textPresenterNm = ''
startDate = '20170101'   
endDate = '20181131'
finalReport = 'recent'
#typesOfBusiness = '1033' ##제조업
typesOfBusiness = ''
#corporationType = 'A' ## A: 코스닥 P: 유가증권시장
closingAccountsMonth = 'all'
#reportName = condition_list
#====================================================
#====================================================
#====================================================

## 1-2 공시 크롤링 함수선언

In [682]:
def dart_crawling_basic(reportName, corporationType):  #corporationType = 'P' ## A: 코스닥 P: 유가증권시장
    ##+++++++++++++++++++++++++++++++++공시 검색 마지막페이지 알아내기 START
    try:
        df_last = []
        df_last = pd.DataFrame(df_last)

        # Curl -> Python 방식
        cookies = {
            'WMONID': 'g9vM2hjgQJ5',
            'JSESSIONID': 'UQbVxoz3quIyiGUo6TvousoCaYIe0TyUEIIOlSX5SgR8A15syM235lrExfZaeIh0.ZG1fZGFydC9kYXJ0MV9kYXJ0X21zMg==',
        }

        headers = {
            'Origin': 'http://dart.fss.or.kr',
            'Accept-Encoding': 'gzip, deflate',
            'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36',
            'Content-Type': 'application/x-www-form-urlencoded',
            'Accept': 'text/html, */*; q=0.01',
            'Referer': 'http://dart.fss.or.kr/dsab002/main.do',
            'X-Requested-With': 'XMLHttpRequest',
            'Connection': 'keep-alive',
        }

        data = [
          ('maxResults', '100'),
          ('maxLinks', maxLinks),
          ('sort', sort),
          ('series', series),
          ('reportNamePopYn', reportNamePopYn),
          ('textCrpNm', textCrpNm),
          ('textPresenterNm', textPresenterNm),
          ('startDate', startDate),
          ('endDate', endDate),
          ('finalReport', finalReport),
          ('typesOfBusiness', typesOfBusiness),  ##제조업
          ('corporationType', corporationType),
          ('closingAccountsMonth', closingAccountsMonth),
          ('reportName', reportName)
        ]

        response = requests.post('http://dart.fss.or.kr/dsab002/search.ax', headers=headers, cookies=cookies, data=data, verify=False)

        dart_data = BeautifulSoup(response.text, 'lxml')
        el_table_navi = dart_data.find( class_="page_list")
        el_td_last = el_table_navi.find("input", alt="맨끝으로")
        pg_last = el_td_last.get('onclick')
        pg_last = pg_last.split('(')[1]
        pg_last = pg_last[:-1]
        pg_last = int(pg_last)
        print('pg_last = ' , pg_last, 'corporationType =',corporationType )
        ##+++++++++++++++++++++++++++++++++공시 검색 마지막페이지 알아내기 END


        try:
            for i in range(1, pg_last + 1) :
                cookies = {
                    'WMONID': 'g9vM2hjgQJ5',
                    'JSESSIONID': 'UQbVxoz3quIyiGUo6TvousoCaYIe0TyUEIIOlSX5SgR8A15syM235lrExfZaeIh0.ZG1fZGFydC9kYXJ0MV9kYXJ0X21zMg==',
                }

                headers = {
                    'Origin': 'http://dart.fss.or.kr',
                    'Accept-Encoding': 'gzip, deflate',
                    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
                    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36',
                    'Content-Type': 'application/x-www-form-urlencoded',
                    'Accept': 'text/html, */*; q=0.01',
                    'Referer': 'http://dart.fss.or.kr/dsab002/main.do',
                    'X-Requested-With': 'XMLHttpRequest',
                    'Connection': 'keep-alive',
                }

                data = [
                  ('currentPage', i),
                  ('maxResults', maxResults),
                  ('maxLinks', maxLinks),
                  ('sort', sort),
                  ('series', series),
                  ('reportNamePopYn', reportNamePopYn),
                  ('textCrpNm', textCrpNm),
                  ('textPresenterNm', textPresenterNm),
                  ('startDate', startDate),
                  ('endDate', endDate),
                  ('finalReport', finalReport),
                  ('typesOfBusiness', typesOfBusiness),  
                  ('corporationType', corporationType),
                  ('closingAccountsMonth', closingAccountsMonth),
                  ('reportName', reportName)
                ]

                response = requests.post('http://dart.fss.or.kr/dsab002/search.ax', headers=headers, cookies=cookies, data=data, verify=False)
                df = pd.read_html(response.text, skiprows=0)[0]
                df_last = df_last.append(df) 
                df_last['keyword'] = reportName
                
            return df_last
        except Exception as e:
            traceback.print_exc()
        return None
    except Exception as e:
        traceback.print_exc()
    return None

In [683]:
## Condition list 의 키워드가 포함된 공시 크롤링 시작  & 테이블 생성 작업

In [684]:
df_dart = []
df_dart = pd.DataFrame(df_dart)
    
for k in range(len(condition_list)):
    print(condition_list[k])
#    df_dart_condition_A = dart_crawling_basic(condition_list[k],'A')
#    df_dart = df_dart.append(df_dart_condition_A)
    df_dart_condition_P = dart_crawling_basic(condition_list[k],'P')
    df_dart = df_dart.append(df_dart_condition_P)

유상증자결정
최대주주


Traceback (most recent call last):
  File "<ipython-input-682-f25421b17487>", line 46, in dart_crawling_basic
    el_td_last = el_table_navi.find("input", alt="맨끝으로")
AttributeError: 'NoneType' object has no attribute 'find'
Traceback (most recent call last):
  File "<ipython-input-682-f25421b17487>", line 46, in dart_crawling_basic
    el_td_last = el_table_navi.find("input", alt="맨끝으로")
AttributeError: 'NoneType' object has no attribute 'find'


투자판단
조회공시


Traceback (most recent call last):
  File "<ipython-input-682-f25421b17487>", line 46, in dart_crawling_basic
    el_td_last = el_table_navi.find("input", alt="맨끝으로")
AttributeError: 'NoneType' object has no attribute 'find'
Traceback (most recent call last):
  File "<ipython-input-682-f25421b17487>", line 46, in dart_crawling_basic
    el_td_last = el_table_navi.find("input", alt="맨끝으로")
AttributeError: 'NoneType' object has no attribute 'find'


단일판매ㆍ공급계약체결
특허권 취득


...

Traceback (most recent call last):
  File "<ipython-input-682-f25421b17487>", line 46, in dart_crawling_basic
    el_td_last = el_table_navi.find("input", alt="맨끝으로")
AttributeError: 'NoneType' object has no attribute 'find'
Traceback (most recent call last):
  File "<ipython-input-682-f25421b17487>", line 46, in dart_crawling_basic
    el_td_last = el_table_navi.find("input", alt="맨끝으로")
AttributeError: 'NoneType' object has no attribute 'find'


타법인 주식 및 출자증권 양수결정
주식소각결정


Traceback (most recent call last):
  File "<ipython-input-682-f25421b17487>", line 46, in dart_crawling_basic
    el_td_last = el_table_navi.find("input", alt="맨끝으로")
AttributeError: 'NoneType' object has no attribute 'find'
Traceback (most recent call last):
  File "<ipython-input-682-f25421b17487>", line 46, in dart_crawling_basic
    el_td_last = el_table_navi.find("input", alt="맨끝으로")
AttributeError: 'NoneType' object has no attribute 'find'


기술도입ㆍ이전ㆍ제휴 계약체결
기타 경영사항


Traceback (most recent call last):
  File "<ipython-input-682-f25421b17487>", line 46, in dart_crawling_basic
    el_td_last = el_table_navi.find("input", alt="맨끝으로")
AttributeError: 'NoneType' object has no attribute 'find'
Traceback (most recent call last):
  File "<ipython-input-682-f25421b17487>", line 46, in dart_crawling_basic
    el_td_last = el_table_navi.find("input", alt="맨끝으로")
AttributeError: 'NoneType' object has no attribute 'find'


In [686]:
df_dart

""


In [674]:
df_dart = df_dart.reset_index(drop=True)
df_dart_customized = df_dart.iloc[:,[1,2,4,6]]

IndexError: positional indexers are out-of-bounds

In [ ]:
df_dart_customized.head()

In [ ]:
## 종목명, 종목코드 가져오기

code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) # 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌
code_df = code_df[['회사명', '종목코드']] # 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다.
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) # 한글로된 컬럼명을 영어로 바꿔준다.
code_df.head()

In [ ]:
code = []
df_dart_customized['접수일자'] = pd.to_datetime(df_dart_customized['접수일자'])
for kk in range(len(df_dart_customized['공시대상회사'])):
    item_name = df_dart_customized.loc[kk,'공시대상회사']
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False).strip()
    df_dart_customized.loc[kk,'code'] = code


In [ ]:
df_dart_customized_process = df_dart_customized

In [ ]:
## 테이블 정리 - 제목에 '기재정정' 포함시 제외, 회사code 못가져오면 제외
#df_dart_customized = df_dart_customized[df_dart_customized.보고서명 in '기재정정']
df_dart_customized_process = df_dart_customized_process[df_dart_customized_process.code != 'Series([], )']
df_dart_customized_process = pd.DataFrame(df_dart_customized_process)
df_dart_customized_process = df_dart_customized_process.reset_index(drop=True)
df_dart_customized_process.head()

In [ ]:
for t in range(len(df_dart_customized_process.index)) :
    datetype = pd.to_datetime(df_dart_customized_process.loc[t,'접수일자'])
    df_dart_customized_process.loc[t,'접수일자'] = datetime.strftime(datetype,"%Y-%m-%d")
    
    if '기재정정' in df_dart_customized_process.loc[t,'보고서명'] :
        df_dart_customized_process.loc[t,'보고서명'] = np.nan
        
        
df_dart_customized_process = df_dart_customized_process.dropna()
df_dart_customized_process = df_dart_customized_process.drop_duplicates()
df_dart_customized_process = df_dart_customized_process.reset_index(drop=True)
df_dart_customized_process

## 주가 가져오기

In [ ]:
''''
import json
from io import StringIO

def get_historical_data(code, start_date) :

    try:
        
        # 기업 인베스팅닷컴 고유 아이디를 구하기
        url = 'https://www.investing.com/search/service/search'
        headers = {
                'User-Agent':'Mozilla',
                'X-Requested-With':'XMLHttpRequest',
            }

        datas = {
            'search_text': code,
            'term': code,
            'country_id': '0',
            'tab_id': 'All'
            }
        res = requests.post(url, data=datas, headers=headers)


        # 해당 데이에서 curr_id 구하기
        code_dict = json.loads(res.text)

        curr_id = str(list(filter(lambda x : x['symbol'] == code, code_dict['All']))[0]['pair_ID'])

       # today = datetime.strftime(datetime.utcnow() + timedelta(hours = 9), "%m/%d/%Y")
        end_date = datetime.strftime(pd.to_datetime(start_date)  + pd.DateOffset(days=14), "%m/%d/%Y")


        # 주가가져오기

        url_data = 'https://www.investing.com/instruments/HistoricalDataAjax'

        datas = {    
                    'curr_id': curr_id,   # 앞에서 구한 기업 고유 아이디
                    'st_date': start_date,  # 함수에서 입력받은 날짜
                  #  'end_date': today,   # 오늘날짜(변경가능)
                    'end_date': end_date,   
                    'interval_sec':'Daily',
                    'sort_col':'date',
                    'sort_ord':'ASC',
                    'action':'historical_data'
                }

        headers = {
                'User-Agent':'Mozilla',
                'X-Requested-With':'XMLHttpRequest',
                }

        res_data = requests.post(url_data, data=datas, headers=headers)
        res_data_pd = pd.read_html(res_data.text)[0]    # 데이터 데이블 받기
        res_data_pd = pd.DataFrame(res_data_pd)

        number_of_row = res_data_pd.shape[0]
        last_row_index= number_of_row -1

        a = res_data_pd.loc[last_row_index,'Price'] # 공시날의 주가
        b = res_data_pd.loc[0,'Price'] # 공시 한달 후 주가
        c = datetime.strftime(pd.to_datetime(res_data_pd.loc[0,'Date']), "%m/%d/%Y")  # 공시 한달 후 날짜
        
        print(code)

        abc = pd.DataFrame({"주가":[a],"한달후주가":[b],"한달후날짜":[c]} )
        return abc
    except Exception as e:
        traceback.print_exc()
    return None
''''

In [ ]:
''''
historical_data_blank = []
historical_data_blank = pd.DataFrame(historical_data_blank)

for tt in range(len(df_dart_customized_process.index)) :
    historical_data_blank = []
    historical_data_blank = get_historical_data(df_dart_customized_process.loc[tt,'code'], df_dart_customized_process.loc[tt,'접수일자']) 
    if historical_data_blank is not None :
        df_dart_customized_process.loc[tt,'주가'] = historical_data_blank.주가[0]
        df_dart_customized_process.loc[tt,'한달후주가'] = historical_data_blank.한달후주가[0]
        df_dart_customized_process.loc[tt,'한달후날짜'] = historical_data_blank.한달후날짜[0]
''''

In [ ]:
import json
from io import StringIO

def get_historical_data(code, start_date) :

    try:
        
        # 기업 인베스팅닷컴 고유 아이디를 구하기
        url = 'https://www.investing.com/search/service/search'
        headers = {
                'User-Agent':'Mozilla',
                'X-Requested-With':'XMLHttpRequest',
            }

        datas = {
            'search_text': code,
            'term': code,
            'country_id': '0',
            'tab_id': 'All'
            }
        res = requests.post(url, data=datas, headers=headers)


        # 해당 데이에서 curr_id 구하기
        code_dict = json.loads(res.text)

        curr_id = str(list(filter(lambda x : x['symbol'] == code, code_dict['All']))[0]['pair_ID'])

       # today = datetime.strftime(datetime.utcnow() + timedelta(hours = 9), "%m/%d/%Y")
        end_date = datetime.strftime(pd.to_datetime(start_date)  + pd.DateOffset(years=1), "%m/%d/%Y")


        # 주가가져오기

        url_data = 'https://www.investing.com/instruments/HistoricalDataAjax'

        datas = {    
                    'curr_id': curr_id,   # 앞에서 구한 기업 고유 아이디
                    'st_date': start_date,  # 함수에서 입력받은 날짜
                  #  'end_date': today,   # 오늘날짜(변경가능)
                    'end_date': end_date,   
                    'interval_sec':'Daily',
                    'sort_col':'date',
                    'sort_ord':'ASC',
                    'action':'historical_data'
                }

        headers = {
                'User-Agent':'Mozilla',
                'X-Requested-With':'XMLHttpRequest',
                }

        res_data = requests.post(url_data, data=datas, headers=headers)
        res_data_pd = pd.read_html(res_data.text)[0]    # 데이터 데이블 받기
        res_data_pd = pd.DataFrame(res_data_pd)

      
        return res_data_pd
    except Exception as e:
        traceback.print_exc()
    return None


In [ ]:
start_date = '01/01/2017'
code = '005930'

In [ ]:
price_table = get_historical_data(code, start_date)
price_table['Date'] = pd.to_datetime(price_table['Date'])
price_r = price_table[::-1]
price_r.to_csv("sample_curprice_SAMSUNG.csv", mode='w', encoding='euckr')

In [ ]:
price_r = price_r.reset_index(drop=True)
price_r = price_r.rename(columns={'Date': '접수일자', 'Price' : '주가'}) # 한글로된 컬럼명을 영어로 바꿔준다.
price_r.head()

In [675]:
for o in range(len(price_r.index)-20):
    p = o + 20
    price_r.loc[o,'한달후날짜'] = price_r.loc[p, '접수일자']
    price_r.loc[o,'한달후주가'] = price_r.loc[p, '주가']

In [676]:
df_dart_customized_process['접수일자'] = pd.to_datetime(df_dart_customized_process['접수일자'])
price_r['접수일자'] = pd.to_datetime(price_r['접수일자'])

dart = pd.merge(df_dart_customized_process, price_r,on = '접수일자')

In [677]:
dart = dart.iloc[:,[0,4,1,2,3,5,11,12]]
dart = dart.reset_index(drop=True)

In [678]:
dart.head()

,공시대상회사,code,보고서명,접수일자,keyword,주가,한달후날짜,한달후주가
0,삼성전자,005930,최대주주등소유주식변동신고서,2018-03-30,최대주주,49220,2018-04-27,53000.0
1,삼성전자,005930,최대주주등소유주식변동신고서,2018-03-12,최대주주,49740,2018-04-09,49200.0
2,삼성전자,005930,최대주주등소유주식변동신고서,2018-02-14,최대주주,49000,2018-03-19,50740.0
3,삼성전자,005930,최대주주등소유주식변동신고서,2018-01-12,최대주주,48200,2018-02-09,44700.0
4,삼성전자,005930,매출액또는손익구조30%(대규모법인은15%)이상변경,2018-01-31,매출액,49900,2018-03-05,45200.0


In [679]:
## 주가상승여부 Check
pos_neg = []
  
for ii in range(len(dart.index)) :

  if dart.loc[ii,'한달후주가'] - dart.loc[ii,'주가'] > 0 :
    pos_neg.append("1")
  else :
    pos_neg.append("0")
   
pos_neg = pd.DataFrame(pos_neg)
pos_neg.columns = ['주가상승여부']

dart['주가상승여부'] = pos_neg['주가상승여부']
dart.head()

,공시대상회사,code,보고서명,접수일자,keyword,주가,한달후날짜,한달후주가,주가상승여부
0,삼성전자,005930,최대주주등소유주식변동신고서,2018-03-30,최대주주,49220,2018-04-27,53000.0,1
1,삼성전자,005930,최대주주등소유주식변동신고서,2018-03-12,최대주주,49740,2018-04-09,49200.0,0
2,삼성전자,005930,최대주주등소유주식변동신고서,2018-02-14,최대주주,49000,2018-03-19,50740.0,1
3,삼성전자,005930,최대주주등소유주식변동신고서,2018-01-12,최대주주,48200,2018-02-09,44700.0,0
4,삼성전자,005930,매출액또는손익구조30%(대규모법인은15%)이상변경,2018-01-31,매출액,49900,2018-03-05,45200.0,0
